# Imports

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd 
import os

/home/szelee/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from datetime import datetime as dt
import pytz

/home/szelee/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
PATH = 'data/nyc-taxi/'
CSV_PATH = 'submissions/v10_ensemble_08trees_54M_rows/'

# Helper functions

In [5]:
def split_df(df, y_fld, subset=None, random_state=42):
    
    if subset: df = df.sample(n=subset, random_state=random_state)

    y = df[y_fld].values
    df.drop(y_fld, axis=1, inplace=True)
    
    return df, y

# Load data

In [6]:
seed = 42 # random seed for replication

In [7]:
train_df = pd.read_feather('tmp/taxi-train-v4-chkpt5')
test_df = pd.read_feather('tmp/taxi-test-v4-chkpt5')
test_df_raw = pd.read_csv(f'{PATH}test.csv', usecols=['key'])

In [8]:
train_df.shape, test_df.shape

((54040753, 24), (9914, 23))

In [9]:
%%time
# subsample = 50_000_000

# X_train, y_train = split_df(train_df, 'fare_amount', subset=subsample, random_state=seed)

X_train, y_train = split_df(train_df, 'fare_amount')

CPU times: user 1.34 s, sys: 2.54 s, total: 3.89 s
Wall time: 3.87 s


In [10]:
X_train.shape, y_train.shape, test_df.shape

((54040753, 23), (54040753,), (9914, 23))

# Batch fit

In [11]:
outdir = CSV_PATH
os.makedirs(outdir, exist_ok=True)
preds = []
n_forest=20
tz = pytz.timezone('Asia/Singapore')

In [12]:
%%time
for i in range (n_forest):
    
    new_seed = i + 1001
    
    print(f"Random Forest [{i+1}/{n_forest}] of 08 trees with random seed {new_seed}")
    m = RandomForestRegressor(n_estimators=8, min_samples_leaf=3, max_features=0.5, n_jobs=-1, random_state=new_seed)
    
    print("Fitting on training data...")
    m.fit(X_train, y_train)

    print("Predicting on test data...")
    test_y_predictions = m.predict(test_df)
    preds.append(test_y_predictions)      
    
    print("Creating submission file...")
    
    cur_dt = dt.now(tz).strftime("%Y%m%d_%H%M%S")
    desc = 'v10_RF_' # description of submission for reference
    ensemb_desc = '_01x08_54M'
    subm_fn =  desc + cur_dt + ensemb_desc + '.csv'
    subm_path = f'{outdir}/{subm_fn}'

    # Write the predictions to a CSV file which we can submit to the competition.
    submission = pd.DataFrame(
        {'key': test_df_raw.key, 'fare_amount': test_y_predictions},
        columns = ['key', 'fare_amount'])
    submission.to_csv(subm_path, index = False)
    
    print(f"Submission: {subm_fn}")
    print()

Random Forest [1/20] of 08 trees with random seed 1001
Fitting on training data...
Predicting on test data...
Creating submission file...
Submission: v10_RF_20180805_034656_01x08_54M.csv

Random Forest [2/20] of 08 trees with random seed 1002
Fitting on training data...
Predicting on test data...
Creating submission file...
Submission: v10_RF_20180805_044312_01x08_54M.csv

Random Forest [3/20] of 08 trees with random seed 1003
Fitting on training data...
Predicting on test data...
Creating submission file...
Submission: v10_RF_20180805_055331_01x08_54M.csv

Random Forest [4/20] of 08 trees with random seed 1004
Fitting on training data...
Predicting on test data...
Creating submission file...
Submission: v10_RF_20180805_065805_01x08_54M.csv

Random Forest [5/20] of 08 trees with random seed 1005
Fitting on training data...
Predicting on test data...
Creating submission file...
Submission: v10_RF_20180805_080651_01x08_54M.csv

Random Forest [6/20] of 08 trees with random seed 1006
Fitti

KeyboardInterrupt: 

In [ ]:
# test_y_predictions = np.mean(np.vstack([preds]),axis=0)

# Results